In [2]:
import csv
import os
from math import ceil

import numpy as np
from PIL import Image, ImageFont, ImageDraw, ImageOps
from matplotlib import pyplot as plt

In [3]:
Tibet_alphabet = ["0F00", "0F01", "0F02", "0F03", "0F04", "0F05", "0F06", "0F07", "0F08", "0F09", "0F0A",
                           "0F0B", "0F0C", "0F0D", "0F40", "0F41", "0F42", "0F43", "0F44", "0F45", , "0F46", "0F47", "0F49", "0F4A"]
tibet_letters = [chr(int(letter, 16)) for letter in Tibet_alphabet]

unit_size = 52
base_threshold = 75

WHITE = 255

In [4]:
def _simple_binarization(img, threshold=base_threshold):
    semitoned = (0.3 * img[:, :, 0] + 0.59 * img[:, :, 1] + 0.11 * img[:, :, 2]).astype(np.uint8)
    new_image = np.zeros(shape=semitoned.shape)
    new_image[semitoned > threshold] = WHITE
    return new_image.astype(np.uint8)

In [5]:
def generate_letters(sin_letters):
    font = ImageFont.truetype("input/Unicode.ttf", unit_size)
    os.makedirs("output/letters", exist_ok=True)
    os.makedirs("output/inverse_letters", exist_ok=True)

    for i in range(len(sin_letters)):
        letter = sin_letters[i]

        width, height = font.getsize(letter)
        img = Image.new(mode="RGB", size=(ceil(width), ceil(height)), color="white")
        draw = ImageDraw.Draw(img)
        draw.text((0, 0), letter, "black", font=font)

        img = Image.fromarray(_simple_binarization(np.array(img), base_threshold), 'L')
        img.save(f"output/letters/{i + 1}.png")

        ImageOps.invert(img).save(f"output/inverse_letters/{i + 1}.png")

In [6]:
def calculate_features(img):
    img_b = np.zeros(img.shape, dtype=int)
    img_b[img != WHITE] = 1 
    (h, w) = img_b.shape
    h_half, w_half = h // 2, w // 2
    quadrants = {
        'top_left': img_b[:h_half, :w_half],
        'top_right': img_b[:h_half, w_half:],
        'bottom_left': img_b[h_half:, :w_half],
        'bottom_right': img_b[h_half:, w_half:]
    }
    weights = {k: np.sum(v) for k, v in quadrants.items()}
    rel_weights = {k: v / (h_half * w_half) for k, v in weights.items()}

    total_pixels = np.sum(img_b) 
    y_indices, x_indices = np.indices(img_b.shape)
    y_center_of_mass = np.sum(y_indices * img_b) / total_pixels
    x_center_of_mass = np.sum(x_indices * img_b) / total_pixels
    center_of_mass = (x_center_of_mass, y_center_of_mass)

    # Normalized center of mass
    normalized_center_of_mass = (x_center_of_mass / (w - 1), y_center_of_mass / (h - 1))

    # Inertia
    inertia_x = np.sum((y_indices - y_center_of_mass) ** 2 * img_b) / total_pixels
    normalized_inertia_x = inertia_x / h ** 2
    inertia_y = np.sum((x_indices - x_center_of_mass) ** 2 * img_b) / total_pixels
    normalized_inertia_y = inertia_y / w ** 2

    return {
        'weight': total_pixels,
        'center_of_mass': center_of_mass,
        'weights': weights,
        'rel_weights': rel_weights,
        'inertia': (inertia_x, inertia_y),
        'normalized_center_of_mass': normalized_center_of_mass,
        'normalized_inertia': (normalized_inertia_x, normalized_inertia_y)
    }

In [7]:
def create_features(sin_letters):
    with open('output/data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['weight', 'weights', 'rel_weights', 'center_of_mass', 'normalized_center_of_mass',
                      'inertia', 'normalized_inertia']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(len(sin_letters)):
            img_src = np.array(Image.open(f'output/letters/{i + 1}.png').convert('L'))
            features = calculate_features(img_src)
            writer.writerow(features)

In [8]:
def create_profiles(sin_letters):
    os.makedirs("output/profiles/x", exist_ok=True)
    os.makedirs("output/profiles/y", exist_ok=True)

    for i in range(len(sin_letters)):
        img = np.array(Image.open(f'output/letters/{i + 1}.png').convert('L'))
        img_b = np.zeros(img.shape, dtype=int)
        img_b[img != WHITE] = 1 

        plt.bar(
            x=np.arange(start=1, stop=img_b.shape[1] + 1).astype(int),
            height=np.sum(img_b, axis=0),
            width=0.9
        )
        plt.ylim(0, unit_size)
        plt.xlim(0, 55)
        plt.savefig(f'output/profiles/x/{i + 1}.png')
        plt.clf()

        plt.barh(
            y=np.arange(start=1, stop=img_b.shape[0] + 1).astype(int),
            width=np.sum(img_b, axis=1),
            height=0.9
        )
        plt.ylim(unit_size, 0)
        plt.xlim(0, 55)
        plt.savefig(f'output/profiles/y/{i + 1}.png')
        plt.clf()

In [12]:
if __name__ == "__main__":
    print("Starting...")
    generate_letters(tibet_letters)
    print("generated letters")

    create_features(tibet_letters)
    print("create marks")

    create_profiles(tibet_letters)
    print("created profiles")

Starting...
generated letters
create marks


C:\Users\vest3\AppData\Local\Temp\ipykernel_20240\3639819545.py:9: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  width, height = font.getsize(letter)


created profiles


<Figure size 640x480 with 0 Axes>